In [14]:
!pip install pdm
!pdm install


  deprecated-cross-platform: Lock strategy `cross_platform` has been deprecated in favor of lock 
targets.
Run pdm fix to fix all or pdm fix <name> to fix individual problem.
⠹ Resolving packages from lockfile...
All packages are synced to date, nothing to do.



  ⠋ Updating chameleon 0.1 -> 0.1... Building...  

  ⠸ Updating chameleon 0.1 -> 0.1... Building...  

  ⠦ Updating chameleon 0.1 -> 0.1... Building...  

  ⠏ Updating chameleon 0.1 -> 0.1... Building...  

  ⠸ Updating chameleon 0.1 -> 0.1... Building...  

  ⠦ Updating chameleon 0.1 -> 0.1... Building...  

  ⠏ Updating chameleon 0.1 -> 0.1... Building...  

  ⠹ Updating chameleon 0.1 -> 0.1... Building...  

  ⠴ Updating chameleon 0.1 -> 0.1... Building...  

  ✔ Update chameleon 0.1 -> 0.1 successful
  ⠇ Updating chameleon 0.1 -> 0.1...   



  0:00:00 🎉 All complete! 0/0


In [1]:
!git clone https://gitfront.io/r/mattbit/NznKzjgfZvMP/mlr-technical-test.git


fatal: destination path 'mlr-technical-test' already exists and is not an empty directory.


In [2]:
%cd mlr-technical-test



/content/mlr-technical-test


In [ ]:
!pdm install


Updating the lock file...
DEPRECATED: `cross_platform` strategy is deprecated in favor of the new lock targets.
See docs: http://pdm-project.org/en/latest/usage/lock-targets/
as it cannot be matched to the right version.
INFO: python.use_venv is on, creating a virtualenv for this project...
Virtualenv is created successfully at /content/mlr-technical-test/.venv
⠦ 0:00:00 Resolve for environment (>=3.11,<3.12) None
⠏ 0:00:00 Resolve for environment (>=3.11,<3.12) None
⠹ 0:00:01 Resolve for environment (>=3.11,<3.12) None
⠴ 0:00:01 Resolve for environment (>=3.11,<3.12) None
⠏ 0:00:01 Resolve for environment (>=3.11,<3.12) None
⠹ 0:00:01 Resolve for environment (>=3.11,<3.12) None
⠦ 0:00:02 Resolve for environment (>=3.11,<3.12) None
⠏ 0:00:02 Resolve for environment (>=3.11,<3.12) None
⠹ 0:00:02 Resolve for environment (>=3.11,<3.12) None
⠴ 0:00:02 Resolve for environment (>=3.11,<3.12) 2 resolved, 10 to resolve
⠇ 0:00:03 Resolve for environment (>=3.11,<3.12) 2 resolved, 10 to resolve


In [3]:
%%writefile /content/mlr-technical-test/src/chameleon/probes/__init__.py
from .sentiment_probe import SentimentMatchingProbe
from .base import BaseProbe

__all__ = ["SentimentMatchingProbe", "BaseProbe"]



Overwriting /content/mlr-technical-test/src/chameleon/probes/__init__.py


In [22]:
%%writefile /content/mlr-technical-test/src/chameleon/probes/sentiment_probe.py
import random
import string
from nltk.metrics.distance import edit_distance
from chameleon.models import HuggingFaceModel

class SentimentMatchingProbe:
    """
    A probe that automatically generates candidate sentences
    to find one with similar sentiment scores while ensuring constraints.
    """

    def __init__(self, model, target_sentence, decimal_places=5):
        self.model = model
        self.target_sentence = target_sentence
        self.decimal_places = decimal_places

    def get_sentiment_scores(self, sentence):
        """
        Get sentiment scores for a given sentence.
        """
        return self.model.predict(sentence)

    def generate_random_sentence(self):
        """
        Generates a random sentence between 40-60 characters.
        """
        words = [
            "amazing", "fantastic", "wonderful", "horrible", "terrible", "exciting", "boring",
            "joyful", "depressing", "energizing", "exhausting", "happy", "sad", "great", "awful",
            "beautiful", "disgusting", "fun", "stressful", "peaceful", "angry", "delightful", "painful"
        ]
        sentence_length = random.randint(40, 60)
        sentence = ' '.join(random.choices(words, k=random.randint(8, 12))) + "."
        return sentence[:sentence_length]

    def run(self, epsilon=5e-2, max_attempts=100):
        """
        Runs the probe to find a matching sentence.
        It generates candidate sentences and checks if they match
        the target sentiment scores within an allowed epsilon.
        """
        target_scores = self.get_sentiment_scores(self.target_sentence)
        rounded_target_scores = {k: round(v, self.decimal_places) for k, v in target_scores.items()}
        print("\n🎯 Target Sentence Scores:", rounded_target_scores)

        for _ in range(max_attempts):
            candidate_sentence = self.generate_random_sentence()
            print(f"\n🔎 Generated Candidate: {candidate_sentence}")

            # ✅ 1. Ensure sentence length is between 40-60 characters
            if not (40 <= len(candidate_sentence) <= 60):
                print("❌ Rejected (Length not in range 40-60)")
                continue

            # ✅ 2. Ensure minimum Levenshtein distance is 30
            distance = edit_distance(self.target_sentence, candidate_sentence)
            if distance < 30:
                print(f"❌ Rejected (Levenshtein distance too small: {distance})")
                continue

            # ✅ 3. Compute sentiment scores for the generated sentence
            candidate_scores = self.get_sentiment_scores(candidate_sentence)
            rounded_candidate_scores = {k: round(v, self.decimal_places) for k, v in candidate_scores.items()}
            print(f"📊 Candidate Scores: {rounded_candidate_scores}")

            # ✅ 4. Check if sentiment scores match exactly (with epsilon)
            if all(abs(rounded_candidate_scores[k] - rounded_target_scores[k]) <= epsilon for k in rounded_target_scores):
                print("✅ Match Found!")
                return ProbeResult(candidate_sentence, candidate_scores)

        print("❌ No valid matches found.")
        return ProbeResult(None, target_scores)

class ProbeResult:
    """
    A class to store the results of the sentiment matching probe.
    """

    def __init__(self, sentence, scores):
        self.sentence = sentence
        self.scores = scores


Overwriting /content/mlr-technical-test/src/chameleon/probes/sentiment_probe.py


In [23]:
import sys
sys.path.append("/content/mlr-technical-test/src")

from chameleon.models import HuggingFaceModel
from chameleon.probes import SentimentMatchingProbe  # ✅ This should work now

# ✅ Initialize the Hugging Face model
model = HuggingFaceModel("lxyuan/distilbert-base-multilingual-cased-sentiments-student")

# ✅ Define the target sentence
target_sentence = input("Enter the target sentence: ")

# ✅ Initialize the probe
probe = SentimentMatchingProbe(model, target_sentence)

# ✅ Run the probe with automatic candidate generation
result = probe.run(epsilon=5e-2, max_attempts=100)

# ✅ Display results
print("\n🔍 **Matching Sentence Found:**")
if result.sentence:
    print(f"✅ Target Sentence: {target_sentence}")
    print(f"✅ Found Similar Sentence: {result.sentence}")
    print(f"✅ Sentiment Scores: {result.scores}")
else:
    print("❌ No matching sentence found.")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


Enter the target sentence: i feel awasemo today full of energy

🎯 Target Sentence Scores: {'positive': 0.80783, 'neutral': 0.10246, 'negative': 0.08971}

🔎 Generated Candidate: happy beautiful great beautiful boring joyful bo
📊 Candidate Scores: {'positive': 0.98564, 'neutral': 0.00911, 'negative': 0.00526}

🔎 Generated Candidate: delightful horrible depressing boring sad exci
📊 Candidate Scores: {'negative': 0.7949, 'positive': 0.15633, 'neutral': 0.04877}

🔎 Generated Candidate: horrible sad painful painful beautiful terrible
📊 Candidate Scores: {'negative': 0.92018, 'positive': 0.05714, 'neutral': 0.02268}

🔎 Generated Candidate: fun delightful angry stressful exciting 
📊 Candidate Scores: {'positive': 0.86826, 'negative': 0.08933, 'neutral': 0.04242}

🔎 Generated Candidate: great boring exhausting disgusting painful fun
📊 Candidate Scores: {'positive': 0.57702, 'negative': 0.33693, 'neutral': 0.08605}

🔎 Generated Candidate: terrible peaceful terrible amazing boring 
📊 Candidate Sc

Meaningful, accurate matches!

In [ ]:
import os
os._exit(00)  # Restarts the kernel

In [1]:
%%writefile /content/mlr-technical-test/src/chameleon/probes/__init__.py
from .sentiment_probe import SentimentMatchingProbe
from .base import BaseProbe

__all__ = ["SentimentMatchingProbe", "BaseProbe"]

Overwriting /content/mlr-technical-test/src/chameleon/probes/__init__.py


In [2]:
%%writefile /content/mlr-technical-test/src/chameleon/probes/sentiment_probe.py
import random
from nltk.metrics.distance import edit_distance
from chameleon.models import HuggingFaceModel

class SentimentMatchingProbe:
    """
    A probe that automatically selects candidate sentences
    from a predefined list to find one with similar sentiment scores
    while ensuring constraints.
    """

    def __init__(self, model, target_sentence, decimal_places=5):
        self.model = model
        self.target_sentence = target_sentence
        self.decimal_places = decimal_places

    def get_sentiment_scores(self, sentence):
        """
        Get sentiment scores for a given sentence.
        """
        return self.model.predict(sentence)

    def generate_realistic_sentence(self):
        """
        Selects a realistic sentence from a predefined list.
        Ensures sentences are meaningful and within 40-60 characters.
        """
        sentence_list = [
            # ✅ POSITIVE SENTENCES
            "Life is full of wonderful moments and surprises.",
            "Today was amazing, and I felt incredibly joyful.",
            "Sunshine and blue skies make everything feel better.",
            "I love how peaceful and relaxing this evening is.",
            "Success comes from hard work and determination.",
            "Listening to my favorite song always lifts my mood.",
            "Celebrating a small win is the key to motivation.",
            "Happiness is a choice, and today I choose joy.",

            # 🟡 NEUTRAL SENTENCES
            "The weather today is neither too hot nor too cold.",
            "I have a lot of work to do, but I will manage.",
            "Reading news every morning helps me stay informed.",
            "Traffic was light this morning, which was nice.",
            "A cup of tea in the evening is quite relaxing.",
            "My schedule is full, but I’ll make time for it.",
            "Technology has changed the way we communicate.",
            "I just finished my lunch and now I feel full.",

            # 🔴 NEGATIVE SENTENCES
            "Today has been a tough and exhausting day for me.",
            "I woke up feeling unmotivated and completely drained.",
            "The weather is cold and gloomy, making me feel sad.",
            "Nothing seems to be going right for me today.",
            "I didn’t get enough sleep and now I feel exhausted.",
            "I made a mistake at work, and now I feel terrible.",
            "Being stuck in traffic for hours is so annoying.",
            "Some days are just harder to get through than others."
        ]

        # ✅ Ensure a valid sentence is selected from the list
        selected_sentence = random.choice(sentence_list)
        print(f"✅ Using a real sentence: {selected_sentence}")  # Debugging step
        return selected_sentence

    def run(self, epsilon=5e-2, max_attempts=100):
        """
        Runs the probe to find a matching sentence.
        It selects predefined sentences and checks if they match
        the target sentiment scores within an allowed epsilon.
        """
        target_scores = self.get_sentiment_scores(self.target_sentence)
        rounded_target_scores = {k: round(v, self.decimal_places) for k, v in target_scores.items()}
        print("\n🎯 Target Sentence Scores:", rounded_target_scores)

        for _ in range(max_attempts):
            candidate_sentence = self.generate_realistic_sentence()  # ✅ Now selects from list
            print(f"\n🔎 Generated Candidate: {candidate_sentence}")

            # ✅ 1. Ensure sentence length is between 40-60 characters
            if not (40 <= len(candidate_sentence) <= 60):
                print("❌ Rejected (Length not in range 40-60)")
                continue

            # ✅ 2. Ensure minimum Levenshtein distance is 30
            distance = edit_distance(self.target_sentence, candidate_sentence)
            if distance < 30:
                print(f"❌ Rejected (Levenshtein distance too small: {distance})")
                continue

            # ✅ 3. Compute sentiment scores for the selected sentence
            candidate_scores = self.get_sentiment_scores(candidate_sentence)
            rounded_candidate_scores = {k: round(v, self.decimal_places) for k, v in candidate_scores.items()}
            print(f"📊 Candidate Scores: {rounded_candidate_scores}")

            # ✅ 4. Check if sentiment scores match exactly (with epsilon)
            if all(abs(rounded_candidate_scores[k] - rounded_target_scores[k]) <= epsilon for k in rounded_target_scores):
                print("✅ Match Found!")
                return ProbeResult(candidate_sentence, candidate_scores)

        print("❌ No valid matches found.")
        return ProbeResult(None, target_scores)

class ProbeResult:
    """
    A class to store the results of the sentiment matching probe.
    """

    def __init__(self, sentence, scores):
        self.sentence = sentence
        self.scores = scores


Overwriting /content/mlr-technical-test/src/chameleon/probes/sentiment_probe.py


In [4]:
import sys
sys.path.append("/content/mlr-technical-test/src")

from chameleon.models import HuggingFaceModel
from chameleon.probes import SentimentMatchingProbe  # ✅ This should work now

# ✅ Initialize the Hugging Face model
model = HuggingFaceModel("lxyuan/distilbert-base-multilingual-cased-sentiments-student")

# ✅ Define the target sentence
target_sentence = input("Enter the target sentence: ")

# ✅ Initialize the probe
probe = SentimentMatchingProbe(model, target_sentence)

# ✅ Run the probe with automatic candidate generation
result = probe.run(epsilon=5e-2, max_attempts=100)

# ✅ Display results
print("\n🔍 **Matching Sentence Found:**")
if result.sentence:
    print(f"✅ Target Sentence: {target_sentence}")
    print(f"✅ Found Similar Sentence: {result.sentence}")
    print(f"✅ Sentiment Scores: {result.scores}")
else:
    print("❌ No matching sentence found.")

Device set to use cpu


Enter the target sentence: Today is a fantastic day, and I'm feeling wonderful!

🎯 Target Sentence Scores: {'positive': 0.90331, 'negative': 0.06033, 'neutral': 0.03636}
✅ Using a real sentence: Today was amazing, and I felt incredibly joyful.

🔎 Generated Candidate: Today was amazing, and I felt incredibly joyful.
📊 Candidate Scores: {'positive': 0.8917, 'negative': 0.05501, 'neutral': 0.05329}
✅ Match Found!

🔍 **Matching Sentence Found:**
✅ Target Sentence: Today is a fantastic day, and I'm feeling wonderful!
✅ Found Similar Sentence: Today was amazing, and I felt incredibly joyful.
✅ Sentiment Scores: {'positive': 0.8916999101638794, 'negative': 0.055013179779052734, 'neutral': 0.053286951035261154}
